<a href="https://colab.research.google.com/github/Kruglikle/-_-/blob/main/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%82%D0%BE%D1%80_%D1%83%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!df -h


Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   59G   50G  55% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  775M  61% /usr/sbin/docker-init
tmpfs           6.4G   36K  6.4G   1% /var/colab
/dev/sda1        85G   66G   20G  78% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [5]:
!pip install transformers datasets torch scikit-learn
!pip install --upgrade pip
!pip install fsspec==2025.3.2


  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [7]:
from google.colab import files
uploaded = files.upload()


Saving spotlight5_tasks_cleaned_final.json to spotlight5_tasks_cleaned_final.json


In [3]:
!pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
!pip install transformers datasets sklearn torch


  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

import shutil

# Указываем путь к файлу на Google Drive
file_id = '1hsJFqSFvQ2ME_Gl8ztmZQPAagjcaH7eL'
file_name = 'cleaned_final.json'

# Путь к файлу на Google Drive
downloaded_file = f"/content/drive/MyDrive/{file_name}"

# Загружаем файл
!gdown --id {file_id} -O {downloaded_file}

# Проверим, что файл загружен
import os
os.path.exists(downloaded_file)


Mounted at /content/drive
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1hsJFqSFvQ2ME_Gl8ztmZQPAagjcaH7eL
To: /content/drive/MyDrive/cleaned_final.json
100% 56.7k/56.7k [00:00<00:00, 105MB/s]


True

In [9]:
import json

# Чтение JSON-файла
with open(downloaded_file, 'r') as file:
    data = json.load(file)

# Посмотрим на структуру данных
print(data[:2])  # выводим первые два элемента для проверки


[{'text': 'put the words alphabetical order', 'label': 'языковое'}, {'text': 'complete the words', 'label': 'языковое'}]


In [10]:
from transformers import BertTokenizer

# Загрузка предобученного токенизатора BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Пример токенизации текста
example_text = data[0]['text']  # предполагаем, что в данных есть поле "text"
tokens = tokenizer(example_text, padding=True, truncation=True, return_tensors="pt")

# Посмотрим на токенизированные данные
print(tokens)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  2404,  1996,  2616, 12440,  7476,  2344,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [12]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']  # предполагаем, что в данных есть поле "text"
        label = item['label']  # предполагаем, что в данных есть поле "label"

        # Токенизация текста
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(),  # Убираем лишнюю размерность
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Создаем датасет
dataset = TextDataset(data, tokenizer)


In [18]:
import torch
from torch.utils.data import Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder

# Инициализация LabelEncoder
label_encoder = LabelEncoder()

class TextDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        # Преобразуем метки в числовые значения
        self.labels = label_encoder.fit_transform([item['label'] for item in data])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']  # предполагаем, что в данных есть поле "text"
        label = self.labels[idx]  # использует преобразованные метки

        # Токенизация текста
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(),  # Убираем лишнюю размерность
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Создаем датасет
dataset = TextDataset(data, tokenizer)

# Загружаем модель для классификации
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))  # динамически определяем количество классов

# Определяем параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Создаем объект Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,  # Можно разделить на тренировочную и тестовую выборки
)

# Начинаем обучение
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.978100
20,0.949000
30,0.899300
40,0.893100
50,0.817200
60,0.861200
70,0.913800
80,0.724300
90,0.758100
100,0.554300


Step,Training Loss
10,0.978100
20,0.949000
30,0.899300
40,0.893100
50,0.817200
60,0.861200
70,0.913800
80,0.724300
90,0.758100
100,0.554300


TrainOutput(global_step=189, training_loss=0.7212271337155942, metrics={'train_runtime': 9677.4051, 'train_samples_per_second': 0.154, 'train_steps_per_second': 0.02, 'total_flos': 393091446085632.0, 'train_loss': 0.7212271337155942, 'epoch': 3.0})

In [23]:
!gdown 1hsJFqSFvQ2ME_Gl8ztmZQPAagjcaH7eL

Downloading...
From: https://drive.google.com/uc?id=1hsJFqSFvQ2ME_Gl8ztmZQPAagjcaH7eL
To: /content/spotlight5_tasks_cleaned_final.json
100% 56.7k/56.7k [00:00<00:00, 22.4MB/s]


In [28]:
import json

with open("spotlight5_tasks_cleaned_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)


In [31]:
from collections import Counter

# Предполагаем, что у каждого объекта есть ключ 'label' с типом упражнения
labels = [item['label'] for item in data]
counter = Counter(labels)

# Печатаем результат
print("📊 Общая статистика по типам упражнений:")
for label, count in counter.items():
    print(f"- {label}: {count}")

# Общий подсчет всех упражнений
total = sum(counter.values())
print(f"\n📘 Всего упражнений в учебнике: {total}")


📊 Общая статистика по типам упражнений:
- языковое: 390
- условно-коммуникативное: 62
- коммуникативное: 46

📘 Всего упражнений в учебнике: 498
